In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pingouin as pg
from scipy import stats
import matplotlib.pyplot as plt
from __future__ import annotations   
from itertools import product  
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.multivariate.manova import MANOVA
from scipy.stats import bootstrap
from scipy.spatial import distance
from numpy.linalg import inv
from scipy.stats import chi2
from normalizato_utile import *

## Manejo que se hizo con el DataFrame original

* Se categorizó la variable MMSE de la siguiente manera:
    * 30-27 puntos: Sin deterioro
    * 26-25 puntos: Dudoso o posible deterioro
    * 24-10 puntos: Deterioro leve a moderado
    * 9-6 puntos: Demencia moderada a severa
    * Menos de 6 puntos: Demencia severa.
* Se categorizó la variable AGE en la siguiente manera:
    * 60-69 años
    * 70-79 años
    * 80-90 años


In [2]:
df = pd.read_csv("Dataset_G2.csv")
pos = 0
# Agrupamiento de edades
for data in df["Age"]:
    if data < 70:
        df["Age"][pos] = "60-69"
    elif data >= 70 and data < 80:
        df["Age"][pos] = "70-79"
    elif data >= 80 and data <= 90:
        df["Age"][pos] = "80-90"
    pos += 1
df["Age"] = df["Age"].astype("category")

# Agrupamiento de MMSE
pos = 0
for data in df["MMSE"]:
    if data >= 27:
        df["MMSE"][pos] = "Sin deterioro"
    elif data >= 25 and data < 27:
        df["MMSE"][pos] = "Posible deterioro"
    elif data >= 10 and data < 25:
        df["MMSE"][pos] = "Deterioro leve"
    elif data >= 6 and data < 10:
        df["MMSE"][pos] = "Deterioro moderado"
    elif data < 6:
        df["MMSE"][pos] = "Demencia severa"
    pos += 1
df["MMSE"] = df["MMSE"].astype("category")
print(df["Age"].value_counts())
print(df["MMSE"].value_counts())

#Eliminar filas con datos faltantes
#print(len(df))
#df = df.dropna()
#print(len(df))
#print(df["Age"].value_counts())
#print(df["MMSE"].value_counts())
df.to_csv(Path("Base_plena.csv", index=False))

C:\Users\user\AppData\Local\Temp\ipykernel_18624\3643071408.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Age"][pos] = "70-79"
C:\Users\user\AppData\Local\Temp\ipykernel_18624\3643071408.py:8: SettingWithCopyWarning: 
A value is tr

Age
80-90    88980
70-79    81101
60-69    80856
Name: count, dtype: int64
MMSE
Deterioro leve        125082
Demencia severa        50317
Deterioro moderado     33688
Sin deterioro          25041
Posible deterioro      16809
Name: count, dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_18624\3643071408.py:38: DeprecationWarning: support for supplying keyword arguments to pathlib.PurePath is deprecated and scheduled for removal in Python 3.14
  df.to_csv(Path("Base_plena.csv", index=False))
